<a href="https://colab.research.google.com/github/swarnavabose2001/Important_Projects/blob/main/Copy_of_HousePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import xgboost as xgb




In [ ]:
# Load training and test data
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')


In [ ]:
# Display the first few rows of the training data
print(train_data.head())

# Check for missing values
print(train_data.isnull().sum())


   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities  ... PoolArea PoolQC Fence MiscFeature MiscVal MoSold  \
0         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
1         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      5   
2         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      9   
3         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
4         Lvl    AllPub  ...        0    NaN   NaN         NaN       0     12   

  YrSold  SaleType  SaleCondition  SalePrice  
0   2008        WD   

In [ ]:
# Fill missing values for numerical features only
for col in train_data.select_dtypes(include=np.number).columns:
    train_data[col].fillna(train_data[col].median(), inplace=True)
for col in test_data.select_dtypes(include=np.number).columns:
    test_data[col].fillna(test_data[col].median(), inplace=True)

# Fill missing values for categorical features with the most frequent value
for col in train_data.select_dtypes(include=['object']).columns:
    train_data[col].fillna(train_data[col].mode()[0], inplace=True)
for col in test_data.select_dtypes(include=['object']).columns:
    test_data[col].fillna(test_data[col].mode()[0], inplace=True)


# Convert categorical variables to dummy variables
train_data = pd.get_dummies(train_data, drop_first=True)
test_data = pd.get_dummies(test_data, drop_first=True)

# Align the train and test dataframes
train_data, test_data = train_data.align(test_data, join='left', axis=1, fill_value=0)

<ipython-input-28-6892ad6a1c9c>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data[col].fillna(train_data[col].median(), inplace=True)
<ipython-input-28-6892ad6a1c9c>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=Tr

In [ ]:
# Define features and target variable
X = train_data.drop('SalePrice', axis=1)  # Features
y = train_data['SalePrice']  # Target variable


In [ ]:
# Example: Creating interaction features
X['OverallQual_LotArea'] = X['OverallQual'] * X['LotArea']  # Example interaction
# Generate polynomial features
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X)

In [ ]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:

# Define the model
model = RandomForestRegressor(random_state=42)

# Define hyperparameters to tune
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Set up Grid Search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid,
                           cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

# Fit Grid Search
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print(f'Best Parameters: {best_params}')

# Use the best estimator
best_model = grid_search.best_estimator_



Fitting 5 folds for each of 24 candidates, totalling 120 fits
Best Parameters: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}


In [ ]:


# Initialize the XGBoost model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Fit the model
xgb_model.fit(X_train, y_train)

# Make predictions
y_pred_xgb = xgb_model.predict(X_val)

# Evaluate the model
mse_xgb = mean_squared_error(y_val, y_pred_xgb)
rmse_xgb = np.sqrt(mse_xgb)
print(f'XGBoost RMSE: {rmse_xgb}')


XGBoost RMSE: 29423.714648995716


In [ ]:


# Perform cross-validation
cv_scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')

# Convert to RMSE
rmse_cv_scores = np.sqrt(-cv_scores)

print(f'Cross-Validation RMSE: {rmse_cv_scores.mean()} ± {rmse_cv_scores.std()}')


Cross-Validation RMSE: 30109.568826315517 ± 4097.372179315068


In [ ]:
# Example: Removing outliers based on SalePrice
q1 = train_data['SalePrice'].quantile(0.25)
q3 = train_data['SalePrice'].quantile(0.75)
iqr = q3 - q1

# Define bounds
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

# Remove outliers
train_data_filtered = train_data[(train_data['SalePrice'] >= lower_bound) & (train_data['SalePrice'] <= upper_bound)]


In [ ]:
# Make predictions
model.fit(X_train, y_train)
y_pred = model.predict(X_val)

# Evaluate the model using Mean Squared Error
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error: {rmse}')



Root Mean Squared Error: 28770.52101354722


In [ ]:
mean_sale_price = np.mean(y)
percentage_error = (rmse / mean_sale_price) * 100
print(f'Percentage of RMSE: {percentage_error:.2f}%')


Percentage of RMSE: 15.90%


In [ ]:
# Make predictions on the test set
# Remove 'SalePrice' from the test_data before prediction
# Assuming 'OverallQual' and 'LotArea' are your original features
# Create the 'OverallQual_LotArea' feature in your test data

test_data['OverallQual_LotArea'] = test_data['OverallQual'] * test_data['LotArea']

test_predictions = model.predict(test_data.drop(columns=['SalePrice']))

# Prepare the submission DataFrame
submission = pd.DataFrame({'Id': test_data['Id'], 'SalePrice': test_predictions})

# Save to CSV
submission.to_csv('submission.csv', index=False)



In [ ]:
importances = model.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
print(feature_importance_df)


                 Feature  Importance
4            OverallQual    0.536107
16             GrLivArea    0.104660
245  OverallQual_LotArea    0.065241
14              2ndFlrSF    0.034015
12           TotalBsmtSF    0.027813
..                   ...         ...
124  Exterior1st_ImStucc    0.000000
187         HeatingQC_Po    0.000000
231     MiscFeature_TenC    0.000000
112     RoofMatl_Membran    0.000000
192       Electrical_Mix    0.000000

[246 rows x 2 columns]
